In [1]:
from datetime import date, timedelta
import datetime
import pandas as pd
import numpy as np
from tqdm import tqdm, tnrange

from sklearn.metrics import mean_squared_error
import lightgbm as lgb

import mlflow
import mlflow.sklearn

from config import (
    RAW_DATA_DIR,
    FEATURE_DIR,
    LAG_DICT,
    SLIDING_DICT
)

/Users/liuyu/anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
# solve lightgbm error on MAC
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [3]:
# load data
df_train = pd.read_csv(
    RAW_DATA_DIR+'train.csv', usecols=[1, 2, 3, 4, 5],
    dtype={'onpromotion': bool},
    converters={'unit_sales': lambda u: np.log1p(
        float(u)) if float(u) > 0 else 0},
    parse_dates=["date"],
    skiprows=range(1, 66458909)  # 2016-01-01
)

df_test = pd.read_csv(
    RAW_DATA_DIR+'test.csv', usecols=[0, 1, 2, 3, 4],
    dtype={'onpromotion': bool},
    parse_dates=["date"]  # , date_parser=parser
).set_index(
    ['store_nbr', 'item_nbr', 'date']
)

items = pd.read_csv(
    RAW_DATA_DIR+'items.csv',
).set_index("item_nbr")

stores = pd.read_csv(
    RAW_DATA_DIR+'stores.csv',
).set_index("store_nbr")

transactions_df = pd.read_csv(
    RAW_DATA_DIR+'transactions.csv'
)

transactions_df = pd.read_csv(
    RAW_DATA_DIR+'transactions.csv'
)

oil_df = pd.read_csv(
    RAW_DATA_DIR+'oil.csv',
    parse_dates=['date'])

### Test Period

2017-08-16 to 2017-08-31

In [4]:
test_start = date(2017, 8, 16)
test_end = date(2017,8, 31)

In [5]:
valid_start = test_start - timedelta(16)
while(1):
    if valid_start.weekday() == test_start.weekday():
        break
    valid_start = valid_start-timedelta(days=1)
valid_end = valid_start + timedelta(15)
print('valid starts from {} to {}'.format(valid_start, valid_end))

valid starts from 2017-07-26 to 2017-08-10


### Valid Period

Considering the more nearer peiods of sales data may have more in common, it would be better to find the nearest period as valid period.

Based on the analysis before, we assume the sales data is periodically with the frequency of 7 days, so we want to keep that feature same
in the train, valid and test period.

So finally, we choose valid period:

2017-07-26 to 2017-08-10


In [6]:
valid_start = date(2017, 7, 26)
valid_end = date(2017, 8, 10)

### Filter Period

#### Earthquake happended on April 16, 2016. It may affect for the next several weeks.

In [7]:
# filter the period which is affected by earthquake.
filter_date = date(2016,4,16) + timedelta(7*4)
lag_max = 140
train_start=  filter_date+timedelta(days=lag_max)

while(1):
    train_start = train_start + timedelta(1)
    if train_start.weekday() == valid_start.weekday():
        break
print('train datasets starts from {}'.format(train_start))

train datasets starts from 2016-10-05


In [8]:
train_start = date(2017,2,8)

### Wages in the public sector are paid every two weeks on the 15 th and on the last day of the month. Supermarket sales could be affected by this.


In [9]:
df_train = df_train[df_train['date']>=filter_date]

/Users/liuyu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  """Entry point for launching an IPython kernel.


#### Promo feature

In [10]:
promo_train = df_train.set_index(
    ["store_nbr", "item_nbr", "date"])[["onpromotion"]]

# missing onpromotions filling
promo_train = promo_train.unstack(level=-1).fillna(False)
promo_train.columns = promo_train.columns.get_level_values(1)

In [11]:
# missing test onpromotions filling
promo_test = df_test[["onpromotion"]].unstack(level=-1).fillna(False)
promo_test.columns = promo_test.columns.get_level_values(1)
# filter those items/stores in promo_test but not in promo_train
promo_test = promo_test.reindex(promo_train.index).fillna(False)

In [12]:
promo_features = pd.concat([promo_train, promo_test], axis=1)
del promo_test, promo_train

In [13]:
transactions_df = pd.read_csv(
    RAW_DATA_DIR+'transactions.csv'
)

## Label

In [14]:
# label
df_train = df_train.set_index(["store_nbr", "item_nbr", "date"])[["unit_sales"]].unstack(level=-1).fillna(0)
# tmp = df_train.set_index(["store_nbr", "item_nbr", "date"])[["unit_sales"]].unstack(level=-1).fillna(0)

## Transactions

In [16]:
transactions_df['date'] = pd.to_datetime(transactions_df['date'])
transactions_df = transactions_df.set_index(["store_nbr", "date"])[["transactions"]].unstack(level=-1).fillna(0)
transactions_df = transactions_df.reindex(df_train.index.get_level_values(0))
transactions_df.columns = transactions_df.columns.get_level_values(1)

## Oil

In [45]:
oil_df = pd.read_csv(
    RAW_DATA_DIR+'oil.csv',
    parse_dates=['date'])

In [46]:
oil_df = oil_df.fillna(oil_df.dcoilwtico.mean())

In [52]:
oil_df.head()

            date      
dcoilwtico  2013-01-01    67.714366
            2013-01-02    93.140000
            2013-01-03    92.970000
            2013-01-04    93.120000
            2013-01-07    93.200000
dtype: float64

In [48]:
oil_df = oil_df.set_index(["date"])

In [51]:
oil_df = oil_df.unstack(level=0)

In [43]:
oil_df = oil_df.set_index(["date"])[["dcoilwtico"]].unstack(level=0).fillna(0)

In [44]:
oil_df.head()

            date      
dcoilwtico  2013-01-01    67.714366
            2013-01-02    93.140000
            2013-01-03    92.970000
            2013-01-04    93.120000
            2013-01-07    93.200000
dtype: float64

In [35]:
oil_df.columns = oil_df.columns.get_level_values(1)

AttributeError: 'Series' object has no attribute 'columns'

In [ ]:
# # oil_df
# oil_df = oil_df.reindex(df_train.index.get_level_values(0))
# oil_df.columns = oil_df.columns.get_level_values(1)

## Item

In [17]:
items = items.reindex(df_train.index.get_level_values(1))

#### Item Family

In [18]:
items['family'] = items['family'].astype('category')
item_family_features = items.family.cat.codes.values

#### Item's class

In [46]:
items['class'] = items['class'].astype('category')
item_class_features = items['class'].cat.codes.values

## Store

In [47]:
stores = stores.reindex(df_train.index.get_level_values(0))

#### Store's city

In [48]:
stores['city'] = stores['city'].astype('category')
store_city_features = stores['city'].cat.codes.values

#### Store's state

In [49]:
stores['state'] = stores['state'].astype('category')
store_state_features = stores['state'].cat.codes.values

#### Store's type

In [50]:
stores['type'] = stores['type'].astype('category')
store_type_features = stores['type'].cat.codes.values

#### Store's cluster

In [51]:
stores['cluster'] = stores['cluster'].astype('category')
store_cluster_features = stores['cluster'].cat.codes.values

In [52]:
df_train.columns = df_train.columns.get_level_values(1)

#### Filling missing date

In [53]:
date_list = df_train.columns
obj_list = pd.date_range(filter_date, test_start-timedelta(1))
diff_list = list(set(obj_list) - set(date_list)) 
for i in diff_list:
    print(i)
    df_train[i] = 0

2016-12-25 00:00:00


In [54]:
date_list = promo_features.columns
obj_list = pd.date_range(filter_date, test_end)
diff_list = list(set(obj_list) - set(date_list)) 
for i in diff_list:
    print(i)
    promo_features[i] = 0

2016-12-25 00:00:00


#### Lagging and sliding windows

In [55]:
LAG_DICT = {'unit_sales': [1,2,3,4,5,6,7,14,21,28,35,42,49,56,63],
            'onpromotion': [2, 3,4,5,6, 7, 14, 21],
            'transactions': [1, 2, 3, 4, 5, 6, 7, 14, 21]}

SLIDING_DICT = {'unit_sales': [3, 4, 5, 6, 7, 14, 21, 30, 60, 63]}



# initialise dirs
RAW_DATA_DIR = 'datasets/'

In [57]:
def get_timespan(df, 
                 start_time,
                 minus,
                 periods,
                 freq='D'):
    return df[pd.date_range(start_time - timedelta(days=minus), periods=periods, freq=freq)]

def gen_dataset(df,
                promo_features,
                item_family_features,
                item_class_features,
                store_city_features,
                store_state_features,
                store_type_features,
                store_cluster_features,
                transactions_df,
                start_time,
                is_train=True):
    # init
    X = pd.DataFrame()

    for i in LAG_DICT['unit_sales']:
        X['lag_{}_sales'.format(i)] = get_timespan(df, start_time, i, 1).values.ravel()
    
    for i in LAG_DICT['onpromotion']:
        X['sum_{}_promo'.format(i)] = get_timespan(promo_features, start_time, i, 1).sum(axis=1).ravel()

    for i in SLIDING_DICT['unit_sales']:
        X["mean_{}_sales".format(i)] = get_timespan(df, start_time, i, i).mean(axis=1).values
        X["std_{}_sales".format(i)] = get_timespan(df, start_time, i, i).std(axis=1).values

    for i in range(7):
        X['mean_4_dow{}_2017'.format(i)] = get_timespan(df, start_time, 28-i, 4, freq='7D').mean(axis=1).values
        X['mean_20_dow{}_2017'.format(i)] = get_timespan(df, start_time, 140-i, 20, freq='7D').mean(axis=1).values

    for i in LAG_DICT['transactions']:
        X['lag_{}_transactions'.format(i)] = get_timespan(transactions_df, start_time, i, 1).values.ravel()

    # for the next to-predict 16 days 
    for i in range(16):
        X["promo_{}".format(i)] = promo_features[start_time + timedelta(days=i)].values.astype(np.uint8)

    X['item_family_features'] = item_family_features

    X['item_class_features'] = item_class_features

    X['store_city_features'] = store_city_features

    X['store_state_features'] = store_state_features

    X['store_type_features'] = store_type_features

    X['store_cluster_features'] = store_cluster_features
        
    if is_train:
        y = df[pd.date_range(start_time, periods=16)].values
        return X, y
    return X


#### Generate train, valid and test sets

In [76]:
print("Preparing dataset...")

nbr_weeks = int((valid_start - train_start).days/7)

X_l, y_l = [], []

for i in tqdm(range(nbr_weeks), desc = 'No. of week'):
    delta = timedelta(days=7 * i)
    X_tmp, y_tmp = gen_dataset(
        df_train,
        promo_features,
        item_family_features,
        item_class_features,
        store_city_features,
        store_state_features,
        store_type_features,
        store_cluster_features,
        transactions_df,
        train_start + delta
    )
    X_l.append(X_tmp)
    y_l.append(y_tmp)
#     break

No. of week:   0%|          | 0/24 [00:00<?, ?it/s]

Preparing dataset...


No. of week: 100%|██████████| 24/24 [00:21<00:00,  1.11it/s]


In [77]:
X_train = pd.concat(X_l, axis=0)
y_train = np.concatenate(y_l, axis=0)
del X_l, y_l

In [78]:
X_val, y_val = gen_dataset(df_train,
                           promo_features,
                           item_family_features,
                           item_class_features,
                           store_city_features,
                           store_state_features,
                           store_type_features,
                           store_cluster_features,
                           transactions_df,
                           valid_start)
X_test = gen_dataset(df_train, 
                    promo_features,
                    item_family_features,
                    item_class_features,
                    store_city_features,
                    store_state_features,
                    store_type_features,
                    store_cluster_features,
                     transactions_df
                    test_start, is_train=False)

TypeError: gen_dataset() missing 1 required positional argument: 'start_time'

#### Train Model

In [30]:
print("Training and predicting models...")
params = {
    'num_leaves': 2**5 - 1,
    'objective': 'regression_l2',
    'max_depth': 8,
    'min_data_in_leaf': 50,
    'learning_rate': 0.05,
    'feature_fraction': 0.75,
    'bagging_fraction': 0.75,
    'bagging_freq': 1,
    'metric': 'l2',
    'num_threads': 4
}

MAX_ROUNDS = 200
val_pred = []
test_pred = []
cate_vars = ['item_family_features',
            'item_class_features',
            'store_city_features',
            'store_state_features',
            'store_type_features',
            'store_cluster_features']

Training and predicting models...


In [31]:
for i in tqdm(range(16)):
    dtrain = lgb.Dataset(
        X_train, label=y_train[:, i],
        categorical_feature=cate_vars,
        weight=pd.concat([items["perishable"]] * nbr_weeks) * 0.25 + 1
    )
    dval = lgb.Dataset(
        X_val, label=y_val[:, i], reference=dtrain,
        weight=items["perishable"] * 0.25 + 1,
        categorical_feature=cate_vars)

    bst = lgb.train(
        params,
        dtrain,
        num_boost_round=MAX_ROUNDS,
#         verbose_eval = False,
        valid_sets=[dtrain, dval], early_stopping_rounds=50)
    print("\n".join(("%s: %.2f" % x) for x in sorted(
        zip(X_train.columns, bst.feature_importance("gain")),
        key=lambda x: x[1], reverse=True
    )))
    val_pred.append(bst.predict(
        X_val, num_iteration=bst.best_iteration or MAX_ROUNDS))
    test_pred.append(bst.predict(
        X_test, num_iteration=bst.best_iteration or MAX_ROUNDS))
    


  0%|          | 0/16 [00:00<?, ?it/s]/Users/liuyu/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:1205: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/Users/liuyu/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


[1]	training's l2: 1.04938	valid_1's l2: 1.00423
Training until validation scores don't improve for 50 rounds.
[2]	training's l2: 0.981164	valid_1's l2: 0.938377
[3]	training's l2: 0.918131	valid_1's l2: 0.877528
[4]	training's l2: 0.862105	valid_1's l2: 0.823612
[5]	training's l2: 0.810182	valid_1's l2: 0.773513
[6]	training's l2: 0.763315	valid_1's l2: 0.728307
[7]	training's l2: 0.720817	valid_1's l2: 0.687407
[8]	training's l2: 0.68249	valid_1's l2: 0.650288
[9]	training's l2: 0.647824	valid_1's l2: 0.617082
[10]	training's l2: 0.616322	valid_1's l2: 0.586896
[11]	training's l2: 0.587809	valid_1's l2: 0.55955
[12]	training's l2: 0.562755	valid_1's l2: 0.535611
[13]	training's l2: 0.539936	valid_1's l2: 0.513771
[14]	training's l2: 0.518578	valid_1's l2: 0.49327
[15]	training's l2: 0.499246	valid_1's l2: 0.474779
[16]	training's l2: 0.481693	valid_1's l2: 0.458007
[17]	training's l2: 0.465802	valid_1's l2: 0.442843
[18]	training's l2: 0.451247	valid_1's l2: 0.428881
[19]	training's 

  6%|▋         | 1/16 [01:33<23:22, 93.47s/it]

[1]	training's l2: 0.94861	valid_1's l2: 0.92514
Training until validation scores don't improve for 50 rounds.
[2]	training's l2: 0.890793	valid_1's l2: 0.868809
[3]	training's l2: 0.838363	valid_1's l2: 0.817604
[4]	training's l2: 0.790858	valid_1's l2: 0.771317
[5]	training's l2: 0.747899	valid_1's l2: 0.729328
[6]	training's l2: 0.709689	valid_1's l2: 0.692348
[7]	training's l2: 0.67449	valid_1's l2: 0.657869
[8]	training's l2: 0.643348	valid_1's l2: 0.627508
[9]	training's l2: 0.614606	valid_1's l2: 0.599446
[10]	training's l2: 0.588337	valid_1's l2: 0.573912
[11]	training's l2: 0.565035	valid_1's l2: 0.551382
[12]	training's l2: 0.543475	valid_1's l2: 0.530368
[13]	training's l2: 0.523972	valid_1's l2: 0.511355
[14]	training's l2: 0.506729	valid_1's l2: 0.494654
[15]	training's l2: 0.490593	valid_1's l2: 0.479024
[16]	training's l2: 0.476019	valid_1's l2: 0.464926
[17]	training's l2: 0.462839	valid_1's l2: 0.4522
[18]	training's l2: 0.450837	valid_1's l2: 0.440511
[19]	training's 

 12%|█▎        | 2/16 [03:04<21:38, 92.76s/it]

...

 88%|████████▊ | 14/16 [20:35<02:55, 87.58s/it]

[1]	training's l2: 1.05903	valid_1's l2: 1.00853
Training until validation scores don't improve for 50 rounds.
[2]	training's l2: 0.994475	valid_1's l2: 0.946353
[3]	training's l2: 0.936067	valid_1's l2: 0.889915
[4]	training's l2: 0.885673	valid_1's l2: 0.84078
[5]	training's l2: 0.84001	valid_1's l2: 0.796495
[6]	training's l2: 0.796148	valid_1's l2: 0.754043
[7]	training's l2: 0.756604	valid_1's l2: 0.715939
[8]	training's l2: 0.720641	valid_1's l2: 0.681266
[9]	training's l2: 0.68995	valid_1's l2: 0.651676
[10]	training's l2: 0.660304	valid_1's l2: 0.623119
[11]	training's l2: 0.635027	valid_1's l2: 0.598858
[12]	training's l2: 0.612237	valid_1's l2: 0.576931
[13]	training's l2: 0.591579	valid_1's l2: 0.557057
[14]	training's l2: 0.571073	valid_1's l2: 0.537718
[15]	training's l2: 0.552456	valid_1's l2: 0.520148
[16]	training's l2: 0.535596	valid_1's l2: 0.504286
[17]	training's l2: 0.520351	valid_1's l2: 0.489746
[18]	training's l2: 0.507666	valid_1's l2: 0.477596
[19]	training's 

 94%|█████████▍| 15/16 [22:02<01:27, 87.48s/it]

[1]	training's l2: 0.957024	valid_1's l2: 0.938696
Training until validation scores don't improve for 50 rounds.
[2]	training's l2: 0.902736	valid_1's l2: 0.885382
[3]	training's l2: 0.853669	valid_1's l2: 0.836832
[4]	training's l2: 0.80938	valid_1's l2: 0.793062
[5]	training's l2: 0.769242	valid_1's l2: 0.753288
[6]	training's l2: 0.732861	valid_1's l2: 0.717433
[7]	training's l2: 0.700055	valid_1's l2: 0.684937
[8]	training's l2: 0.670381	valid_1's l2: 0.65549
[9]	training's l2: 0.644377	valid_1's l2: 0.629698
[10]	training's l2: 0.619915	valid_1's l2: 0.605598
[11]	training's l2: 0.597714	valid_1's l2: 0.583727
[12]	training's l2: 0.577712	valid_1's l2: 0.563918
[13]	training's l2: 0.559637	valid_1's l2: 0.546048
[14]	training's l2: 0.543108	valid_1's l2: 0.529748
[15]	training's l2: 0.52824	valid_1's l2: 0.515055
[16]	training's l2: 0.514728	valid_1's l2: 0.50179
[17]	training's l2: 0.502474	valid_1's l2: 0.489825
[18]	training's l2: 0.491286	valid_1's l2: 0.47894
[19]	training's 

100%|██████████| 16/16 [23:30<00:00, 87.47s/it]


#### Generate submission

In [33]:
mse = mean_squared_error(y_val, np.array(val_pred).transpose())
mse

0.3498481141075822

In [95]:
mse = mean_squared_error(y_val, np.array(val_pred).transpose())

mlflow.set_experiment('grocery forecasting')
with mlflow.start_run(run_name='lgbm'):
    mlflow.log_param('model', 'lgbm')
    mlflow.log_param('train starts', train_start)
    mlflow.log_params(params)
    mlflow.log_param('lagging', LAG_DICT.values())
    mlflow.log_param('slidingWindows', SLIDING_DICT.values())
    mlflow.log_param('item_info', 'Yes')
    mlflow.log_param('store_info', 'Yes')
    mlflow.log_param('private score', 0.52193)
    mlflow.log_param('private rank', '14%')
    mlflow.log_param('public score', 0.51609)

    mlflow.log_metric('mse', mse)
    
print("Validation mse:", mse)

Validation mse: 0.35123858092295934


In [32]:
print("Making submission...")
y_test = np.array(test_pred).transpose()
df_preds = pd.DataFrame(
    y_test, index=df_train.index,
    columns=pd.date_range("2017-08-16", periods=16)
).stack().to_frame("unit_sales")
df_preds.index.set_names(["store_nbr", "item_nbr", "date"], inplace=True)

submission = df_test[["id"]].join(df_preds, how="left").fillna(0)
submission["unit_sales"] = np.clip(np.expm1(submission["unit_sales"]), 0, 1000)
submission.to_csv('lgb.csv', float_format='%.4f', index=None)

Making submission...
